In [1]:
import pandas as pd
import numpy as np
import json, os, math
from collections import defaultdict as ddict

In [2]:
# adding grades for another project just involves changing these and re-running

# file exported from Canvas
file_a = "2021-12-23T1908_Grades-FA21_COMPSCI_220_005.csv"
# new file we'll create to be imported to Canvas
file_b = "canvas-new.csv"

# will update this project and all before it
last_proj = 13

# this directory contains p1.json, p2.json, etc
# grades_dir = ("/Users", "trh", "g", "caraza-harter-com", "tools", "grades")
#grades_dir = ("/Users", "souravp", "Documents", "TA", "Fall20", "caraza-harter-com", "tools", "grades")
grades_dir = ("/home", "caday", "Documents", "caraza-harter-com", "cs220_tools", "grades")

# how many late days can they use before they get ZERO?
late_allowed = 12 # Should be 12, I changed it since we allowed late resubmission

In [3]:
# c is the canvas DataFrame
c = pd.read_csv(file_a)
c.head()

,Student,ID,SIS User ID,SIS Login ID,Section,CS220 Student Information Survey (1350587),Piazza sign-up (1353972),End of semester survey (1407199),Quiz1 (1359150),Quiz2 (1364084),...,Imported Assignments Final Score,Imported Assignments Unposted Final Score,Current Score,Unposted Current Score,Final Score,Unposted Final Score,Current Grade,Unposted Current Grade,Final Grade,Unposted Final Grade
0,Points Possible,NaN,NaN,NaN,NaN,1.0,1.0,1.0,20.00,20.00,...,(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only)
1,"BABLANI, AAYUSH",307065.0,UW907P986,BABLANI2@WISC.EDU,COMP SCI 220:LAB352 and COMP SCI 220:LEC005,1.0,1.0,1.0,20.00,20.00,...,100,100,95.53,95.53,95.53,95.53,A,A,A,A
2,"BARREDO, BIANCA",264028.0,UW828T914,BARREDO@WISC.EDU,COMP SCI 220:LAB351 and COMP SCI 220:LEC005,1.0,1.0,0.0,18.67,15.67,...,50.97,50.97,81.6,81.6,81.6,81.6,B,B,B,B
3,"BENYUKHIS, MITCHELL",289565.0,UW845X249,BENYUKHIS@WISC.EDU,COMP SCI 220:LAB351 and COMP SCI 220:LEC005,1.0,1.0,1.0,20.00,19.75,...,96.11,96.11,92.81,92.81,92.81,92.81,AB,AB,AB,AB
4,"Bi, Peter",385116.0,UW905B940,QBI5@WISC.EDU,COMP SCI 220:LAB351 and COMP SCI 220:LEC005,1.0,1.0,1.0,18.00,19.88,...,48.61,48.61,85.2,85.2,85.2,85.2,B,B,B,B


In [4]:
# c = c.iloc[1:, :].set_index("SIS Login ID")
# c.head()

In [5]:
# c.loc["ABBASBAYLI@WISC.EDU", "Project 1"] = 1
# c.head()
list(c)

['Student',
 'ID',
 'SIS User ID',
 'SIS Login ID',
 'Section',
 'CS220 Student Information Survey (1350587)',
 'Piazza sign-up (1353972)',
 'End of semester survey (1407199)',
 'Quiz1 (1359150)',
 'Quiz2 (1364084)',
 'Quiz 3 (1369248)',
 'Quiz 4 (1373030)',
 'Quiz 5 (1376711)',
 'Quiz 6 (1379791)',
 'Quiz 7 (1383010)',
 'Quiz 8 (1386220)',
 'Quiz 10 (1392443)',
 'Project 1 (1364427)',
 'Project 2 (1364405)',
 'Project 3 (1364406)',
 'Project 4 (1364407)',
 'Project 5 (1364408)',
 'Project 6 (1364410)',
 'Project 7 (1364411)',
 'Project 8 (1364413)',
 'Project 9 (1364414)',
 'Project 10 (1364415)',
 'Project 11 (1364417)',
 'Project 12 (1364421)',
 'Project 13 (1364422)',
 'Exam 1 (1373860)',
 'Exam 2 (1389218)',
 'Final Exam (1410893)',
 'Quiz 11 (1399791)',
 'Quiz 12 (1404508)',
 'Quiz 9 (1392440)',
 'Communication Current Score',
 'Communication Unposted Current Score',
 'Communication Final Score',
 'Communication Unposted Final Score',
 'Quizzes Current Score',
 'Quizzes Unposted 

In [6]:
def proj_col(p):
    pcol = None
    if(p=='final'):
        return 'Final project (833645)'
    for col in list(c):
        if col.startswith("Project "+str(p[1:])+" "):
            pcol = col
    assert(pcol != None)
    return pcol
# proj_col('final')
pcol = proj_col('p13')

# Final column name: Final project (833645)

In [7]:
idx = c["SIS Login ID"].str.split("@", expand=True)[0].str.lower()
idx = idx.rename("")
idx[0] = "points" # FIX this after tweaks
# idx[1] = "canvas_points"
c.set_index(idx, inplace=True)
c.head()

,Student,ID,SIS User ID,SIS Login ID,Section,CS220 Student Information Survey (1350587),Piazza sign-up (1353972),End of semester survey (1407199),Quiz1 (1359150),Quiz2 (1364084),...,Imported Assignments Final Score,Imported Assignments Unposted Final Score,Current Score,Unposted Current Score,Final Score,Unposted Final Score,Current Grade,Unposted Current Grade,Final Grade,Unposted Final Grade
,,,,,,,,,,,,,,,,,,,,,
points,Points Possible,NaN,NaN,NaN,NaN,1.0,1.0,1.0,20.00,20.00,...,(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only),(read only)
bablani2,"BABLANI, AAYUSH",307065.0,UW907P986,BABLANI2@WISC.EDU,COMP SCI 220:LAB352 and COMP SCI 220:LEC005,1.0,1.0,1.0,20.00,20.00,...,100,100,95.53,95.53,95.53,95.53,A,A,A,A
barredo,"BARREDO, BIANCA",264028.0,UW828T914,BARREDO@WISC.EDU,COMP SCI 220:LAB351 and COMP SCI 220:LEC005,1.0,1.0,0.0,18.67,15.67,...,50.97,50.97,81.6,81.6,81.6,81.6,B,B,B,B
benyukhis,"BENYUKHIS, MITCHELL",289565.0,UW845X249,BENYUKHIS@WISC.EDU,COMP SCI 220:LAB351 and COMP SCI 220:LEC005,1.0,1.0,1.0,20.00,19.75,...,96.11,96.11,92.81,92.81,92.81,92.81,AB,AB,AB,AB
qbi5,"Bi, Peter",385116.0,UW905B940,QBI5@WISC.EDU,COMP SCI 220:LAB351 and COMP SCI 220:LEC005,1.0,1.0,1.0,18.00,19.88,...,48.61,48.61,85.2,85.2,85.2,85.2,B,B,B,B


In [8]:
# we'll merge all project grades here, ordered by project number
rows = []

# loop over project JSON files, updating Canvas
for i in range(1, last_proj+1):
    path = grades_dir + ("p%d.json"%i,)
    path = os.path.join(*path)
    with open(path) as f:
        rows += json.load(f)

# path = grades_dir + ("final.json",)
# path = os.path.join(*path)
# with open(path) as f:
#     rows += json.load(f)

In [9]:
c.loc["points"]

Student                       Points Possible
ID                                        NaN
SIS User ID                               NaN
SIS Login ID                              NaN
Section                                   NaN
                                 ...         
Unposted Final Score              (read only)
Current Grade                     (read only)
Unposted Current Grade            (read only)
Final Grade                       (read only)
Unposted Final Grade              (read only)
Name: points, Length: 64, dtype: object

In [10]:
total_late = ddict(int)

# update Canvas grades, cell-by-cell
for row in rows:
    net_id = row["net_id"]
    if not net_id in c.index:
        continue # they dropped?f
    
    pcol = proj_col(row["project"])
#     print(pcol)
    max_points = 100.
#     if(row["project"]=='p9' or row["project"]=='p10'):
#         max_points = 150.
#     if(row["project"]=='final'):
#         max_points = 27.
    possible = float(c.loc["points", pcol])
#     print(possible)
    score = round((row["score"] / max_points) * possible, 2)
    late = row["late_days"]
#     print(row['score'], score)
    
    if late > 0:
        total_late[net_id] += late
        if total_late[net_id] > late_allowed:
            warn = "WARNING: not counting '%s' grade of %d for '%s' due to lateness"
            print(warn % (pcol, score, net_id))
            score = 0

    # update Canvas if new grade is better
    try:
        prev = float(c.loc[net_id, pcol])
    except:
        prev = 0
#     if math.isnan(prev) or score >= prev or score <prev:
#         # update their canvas score
#         c.loc[net_id, pcol] = score
    if math.isnan(prev) or score >= (prev+0.01):
        # increase their canvas score
        c.loc[net_id, pcol] = score
        print("Updated for ", net_id, prev, score, pcol)
    if math.isnan(prev) or score <= (prev-0.01):
        warn = "WARNING: not reducing '%s' grade for '%s' from %f to %f"
        print(warn % (pcol, net_id, prev, score))
    if prev>4.0:
        c.loc[net_id, pcol] = score
        print("Clipped for ", net_id, prev, score, pcol)

Updated for  cttran3 0.2 2.16 Project 11 (1364417)
Updated for  jchen837 0.0 3.8 Project 12 (1364421)
Updated for  hli735 0.0 3.8 Project 12 (1364421)


In [11]:
total_late

defaultdict(int,
            {'xshen99': 6,
             'aatran2': 5,
             'qbi5': 11,
             'sanev': 33,
             'jschweda': 10,
             'bzhang378': 3,
             'hfeng43': 4,
             'emgore': 5,
             'chen2235': 8,
             'hliu649': 5,
             'barredo': 3,
             'jlee888': 9,
             'sheo9': 3,
             'goleb': 2,
             'fwang264': 2,
             'qjin42': 7,
             'saheyer': 3,
             'lee872': 8,
             'xlu323': 9,
             'dskwon2': 9,
             'mrgackstette': 3,
             'zmayhew': 5,
             'kins': 5,
             'jsim24': 2,
             'liu2263': 19,
             'zjia34': 3,
             'wxie48': 2,
             'nrochlin': 7,
             'stewari': 1,
             'hhan76': 2,
             'jchen837': 7,
             'rvyas': 9,
             'sgomez6': 1,
             'hli735': 7,
             'bablani2': 9,
             'yzhang2547': 2,
             '

# apply zeros for cheating

In [12]:
for k in total_late:
    if total_late[k] > 10:
        print(k)

qbi5
sanev
liu2263


In [13]:
def set_proj(df, net_id, proj, score):
    pcol = proj_col(proj)
#     for col in df.columns:
#         if col.split(" ")[0].lower() == proj.lower():
#             break
#     else:
#         raise Exception("could not find {} {}".format(net_id, proj))
    assert net_id in df.index
    df.loc[net_id, pcol] = score
        
with open("cheating.json") as f:
    cheating = json.load(f)
print(cheating)
for proj, students in cheating.items():
    for net_id, score in students.items():
        print("Set score to {} on {} for {}".format(score, proj, net_id))
        set_proj(c, net_id, proj, score)

{'p11': {'etferguson2': 0, 'arpetersen2': 0}, 'p12': {'tsparikh': 0, 'hpatel29': 0}}
Set score to 0 on p11 for etferguson2


AssertionError: 

In [14]:
# create new Canvas CSV that we can import back in
c.to_csv(file_b, index=False)

In [15]:
with pd.option_context('display.max_columns', 40):
    print(c.describe(include='all'))

            Student             ID SIS User ID    SIS Login ID  \
count            90      89.000000          88              89   
unique           90            NaN          88              89   
top     VYAS, RAHUL            NaN   UW862C247  QJEHN@WISC.EDU   
freq              1            NaN           1               1   
mean            NaN  304977.606742         NaN             NaN   
std             NaN   64599.714026         NaN             NaN   
min             NaN  161535.000000         NaN             NaN   
25%             NaN  252208.000000         NaN             NaN   
50%             NaN  306394.000000         NaN             NaN   
75%             NaN  368450.000000         NaN             NaN   
max             NaN  406349.000000         NaN             NaN   

                                            Section  \
count                                            89   
unique                                            3   
top     COMP SCI 220:LAB351 and COMP SCI 2

In [16]:
print("done!")

done!
